## Clone repository and setup environment
Clone the project from GitHub and navigate to the project directory.

In [ ]:
import os
import subprocess

if not os.path.exists('/content/nlp_pj'):
    print('Cloning repository...')
    subprocess.check_call([
        'git', 'clone', 
        'https://github.com/ndh140305/2526I_INT3406_3_Group_17_Seq2SeqTransformer',
        '/content/nlp_pj'
    ])
    print('Repository cloned.')
else:
    print('Repository already exists.')

os.chdir('/content/nlp_pj')
print('Current directory:', os.getcwd())

print('Installing dependencies...')
subprocess.check_call(['pip', 'install', '-q', '-r', 'requirements.txt'])
print('Dependencies installed.')

# Train Transformer (3-epoch run with checkpoints every 3 epochs)
This notebook runs `train_main.py` with the requested config. Checkpoints are saved every 3 epochs and `best_model.pth` is overwritten at each save.

## Download data and checkpoint from shared Drive
Use the shared folder (data.zip + best_model.pth) to populate `data/processed/` and `training/checkpoints/` before training.

In [ ]:
import os, pathlib, subprocess, sys

try:
    import gdown  
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])

folder_id = "1b2Wn_I3m2-2WBPI3H1TtDiR0URazIOsg"
workdir = pathlib.Path('.')

data_dir = workdir / 'data' / 'processed'
data_dir.mkdir(parents=True, exist_ok=True)
ckpt_dir = workdir / 'training' / 'checkpoints'
ckpt_dir.mkdir(parents=True, exist_ok=True)

download_dir = workdir / 'downloads'
download_dir.mkdir(exist_ok=True)

print('Downloading folder from Drive...')

subprocess.check_call([
    sys.executable, '-m', 'gdown', '--folder', '--id', folder_id, '-O', str(download_dir)
])

zip_path = download_dir / 'data.zip'
if zip_path.exists():
    subprocess.check_call(['unzip', '-o', str(zip_path), '-d', str(data_dir)])
    print('Unzipped data.zip into data/processed')
else:
    print('data.zip not found in downloaded folder; please check the shared Drive contents')

# load checkpoint
best_ckpt = download_dir / 'best_model.pth'
if best_ckpt.exists():
    target_ckpt = ckpt_dir / 'best_model.pth'
    target_ckpt.write_bytes(best_ckpt.read_bytes())
    print('best_model.pth copied to training/checkpoints')
else:
    print('best_model.pth not found in downloaded folder; continuing without it')

print('Done downloading assets.')

In [ ]:
import os, sys, pathlib

root = pathlib.Path('.')
print('Current working dir:', root.resolve())

source_path = pathlib.Path('data/processed/train_source_ids.npy')
target_path = pathlib.Path('data/processed/train_target_ids.npy')
print('Source exists:', source_path.exists())
print('Target exists:', target_path.exists())


In [ ]:
from pathlib import Path
import shutil

root = Path('.')
expected_src = root / 'data' / 'processed' / 'train_source_ids.npy'
expected_tgt = root / 'data' / 'processed' / 'train_target_ids.npy'

if not expected_src.exists() or not expected_tgt.exists():
    print('Attempting to locate missing data files and copy into data/processed...')
    search_roots = [root / 'downloads', root / 'data']

    def find_one(name):
        for base in search_roots:
            if base.exists():
                matches = list(base.rglob(name))
                if matches:
                    return matches[0]
        return None

    src_found = expected_src if expected_src.exists() else find_one('train_source_ids.npy')
    tgt_found = expected_tgt if expected_tgt.exists() else find_one('train_target_ids.npy')

    (root / 'data' / 'processed').mkdir(parents=True, exist_ok=True)

    if src_found and not expected_src.exists():
        shutil.copy(src_found, expected_src)
        print('Copied source from', src_found, '->', expected_src)
    if tgt_found and not expected_tgt.exists():
        shutil.copy(tgt_found, expected_tgt)
        print('Copied target from', tgt_found, '->', expected_tgt)

print('Source exists:', expected_src.exists())
print('Target exists:', expected_tgt.exists())

In [ ]:
pad_token_id = 0
source_ids = 'data/processed/train_source_ids.npy'
target_ids = 'data/processed/train_target_ids.npy'
batch_size = 32
epochs = 3
d_model = 256
num_encoder_layers = 4
num_decoder_layers = 4
d_ff = 1024
num_heads = 8

print('Config loaded.')

In [ ]:
import subprocess, textwrap, shlex

cmd = f"python train_main.py \\\n  --pad_token_id {pad_token_id} \\\n  --source_ids {source_ids} \\\n  --target_ids {target_ids} \\\n  --batch_size {batch_size} \\\n  --epochs {epochs} \\\n  --d_model {d_model} \\\n  --num_encoder_layers {num_encoder_layers} \\\n  --num_decoder_layers {num_decoder_layers} \\\n  --d_ff {d_ff} \\\n  --num_heads {num_heads}"

print('Running command:\n', cmd)

process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
stdout, stderr = process.communicate()

print('\n--- STDOUT ---')
print(stdout)
if stderr:
    print('\n--- STDERR ---')
    print(stderr)

print('\nExit code:', process.returncode)

import pathlib
ckpt_dir = pathlib.Path('training/checkpoints')
if ckpt_dir.exists():
    print('Checkpoints:')
    for p in sorted(ckpt_dir.glob('*.pth')):
        print(' -', p)
else:
    print('No checkpoints directory found')

In [ ]:
import pathlib, json
metrics_path = pathlib.Path('training/metrics.json')
if metrics_path.exists():
    data = json.loads(metrics_path.read_text())
    print('Last metrics entry:' if data else 'Metrics file empty')
    if data:
        print(data[-1])
else:
    print('metrics.json not found')

best_model = pathlib.Path('training/checkpoints/best_model.pth')
print('Best model exists:', best_model.exists())